In [17]:
import tensorflow as tf
import numpy as np
import cv2

In [18]:
img_height=720
img_width=1280
#Network Parameters
n_input=img_height*img_width*3

#tf graph input
x=tf.placeholder(tf.float32,[None,n_input])

In [19]:
def getTestPicArray(filename) :
    im = cv2.imread(filename)
    #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_arr = np.array(im)
    nm = im_arr.reshape((1, n_input))
    nm = nm.astype(np.float32)
    return nm

In [20]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,outputShape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=outputShape,strides=[1,1,1,1],padding='SAME'),b))

In [21]:
wc1=tf.Variable(tf.random_normal([5,5,3,16]),name="wc1")
bc1=tf.Variable(tf.random_normal([16]),name="bc1")

wc2=tf.Variable(tf.random_normal([5,5,16,64]),name="wc2")
bc2=tf.Variable(tf.random_normal([64]),name="bc2")

wc3=tf.Variable(tf.random_normal([5,5,64,64]),name="wc3")
bc3=tf.Variable(tf.random_normal([64]),name="bc3")

wc4=tf.Variable(tf.random_normal([5,5,16,64]),name="wc4")
bc4=tf.Variable(tf.random_normal([16]),name="bc4")

wc5=tf.Variable(tf.random_normal([5,5,16,16]),name="wc5")
bc5=tf.Variable(tf.random_normal([16]),name="bc5")



In [22]:
#Construct model
_X=tf.reshape(x,shape=[-1,img_height,img_width,3])
conv1_1=conv2d(_X,wc1,bc1)
conv1_2=max_pool(conv1_1,k=2)

conv2_1=conv2d(conv1_2,wc2,bc2)
conv2_2=max_pool(conv2_1,k=2)

conv3=conv2d(conv2_2,wc3,bc3)
conv3=max_pool(conv3,k=1)

add1=tf.add(conv2_2,conv3)
#add1=conv2+conv3

conv_t1=tf.image.resize_bilinear(add1,conv2_1.get_shape().as_list()[1:3])
tmpShape=conv1_2.get_shape().as_list()
tmpShape[0]=1
conv_t1=conv2d_transpose(conv_t1,wc4,bc4,outputShape=tmpShape)

add2=tf.add(conv1_2,conv_t1)

conv_t2=tf.image.resize_bilinear(add2,conv1_1.get_shape().as_list()[1:3])
tmpShape=_X.get_shape().as_list()
tmpShape[0]=1
tmpShape[3]=16
conv_t2=conv2d_transpose(conv_t2,wc5,bc5,outputShape=tmpShape)

In [23]:
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [24]:
with tf.Session(config=config) as sess:
    sess.run(init)
    imX=getTestPicArray("test.jpg")
    print("get image")
    #print(imX.shape)
    output=sess.run(conv_t2,feed_dict = {x:imX})
    print(output.shape)


get image
(1, 720, 1280, 16)
